# Stable Diffusion Playground

<img src="../images/goose.png" align=left>

# Import Libraries

In [ ]:
import torch, random, gc
from torch import autocast
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

# Define Functions

In [ ]:
def clear_gpu():
    gc.collect()
    torch.cuda.empty_cache()

def generate(prompt, seed=None, guidance_scale=7.1, num_inference_steps=50):
    prompt = prompt
    if seed == None:
        seed = random.randint(0, 999999999999999)
    print(f"Seed: {seed}")
    generator = torch.Generator("cuda").manual_seed(seed)
    with autocast("cuda"):
        image = pipe(prompt, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps, generator=generator)["sample"][0]  
    return seed, image

def from_image(prompt, init_image, seed=None, strength=0.75, guidance_scale=7.1, num_inference_steps=50):
    prompt = prompt
    if seed == None:
        seed = random.randint(0, 999999999999999)
    print(f"Seed: {seed}")
    generator = torch.Generator("cuda").manual_seed(seed)
    with autocast("cuda"):
        image = pipe(init_image, prompt, strength=strength, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps, generator=generator)["sample"][0]  
    return seed, image

# Create Image

In [ ]:
clear_gpu()

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True, revision="fp16", torch_dtype=torch.float16)
pipe = pipe.to(device)

In [ ]:
prompt = "A burning computer, by Salvador Dali, matte painting trending on artstation HQ"
seed, image = generate(prompt, seed=334627248678436, guidance_scale=7.0, num_inference_steps=50)
image

In [ ]:
image.save("input.png")

# From Base Image

In [ ]:
del pipe
clear_gpu()

In [ ]:
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    use_auth_token=True,
    revision="fp16", 
    torch_dtype=torch.float16,
)
pipe = pipe.to(device)

In [ ]:
with open('input.png', 'rb') as img_file:
    byte_content = img_file.read()
init_image = Image.open(BytesIO(byte_content)).convert("RGB")

In [ ]:
prompt = "Artificial intelligence hacker trapped inside a computer writing equations"
seed, image = from_image(init_image, prompt, seed=833456721105365, strength=0.61, guidance_scale=7.1, num_inference_steps=30)
image

In [ ]:
prompt = "A hamburger dinner, oil painting by Jan Van Eyck"
seed, image = from_image(init_image, prompt, seed=seed, strength=0.65, guidance_scale=7.1, num_inference_steps=50)
image

In [ ]:
prompt = "An illustration by Beatrix Potter of two badgers having supper"
seed, image = from_image(init_image, prompt, seed=seed, strength=0.85, guidance_scale=7.5, num_inference_steps=50)
image